In [157]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src.induction_pinn import PINN, numpy_to_tensor, train
from src.utils import get_data_and_feed
from typing import Optional, Union
from scipy.integrate import solve_ivp

import torch
import torch.nn as nn
from tqdm import tqdm 

pd.options.mode.chained_assignment = None

np.set_printoptions(precision=4)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [158]:
def plot_feed(feeds):
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(feeds['Time'], feeds['F'], width=feeds['Duration'], align='edge')
    ax.set_xlabel('Time (h)')
    ax.set_ylabel('Feed (mL/h)')
    ax.set_title('Feed vs Time')
    plt.show()
    
def plot_simulation(t: Optional[list] = None, y: Optional[list] = None, full_df: Optional[pd.DataFrame] = None, train_df: Optional[pd.DataFrame] = None, net_df: Optional[pd.DataFrame] = None, title: Optional[str] = None):
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 10))
    
    if t is not None and y is not None:
        ax1.plot(t, y[0], label='Biomass (ODE)', alpha=0.6)
        ax1.plot(t, y[1], label='Glucose (ODE)', alpha=0.6)
        ax2.plot(t, y[2], label='Protein (ODE)', alpha=0.6)
        ax3.plot(t, y[3], label='Volume (ODE)', alpha=0.6)
    
    if full_df is not None:
        ax1.scatter(full_df['RTime'], full_df['Glucose'], label='Glucose (EXP)', color='red', alpha=0.2)   
        ax1.scatter(full_df['RTime'], full_df['Biomass'], label='Biomass (EXP)', color='green', alpha=0.2)
        ax2.scatter(full_df['RTime'], full_df['Protein'], label='Protein (EXP)', color='blue', alpha=0.2)
        ax3.scatter(full_df['RTime'], full_df['V'], label='Volume (EXP)', color='purple', alpha=0.2)
    
    if train_df is not None:
        ax1.scatter(train_df['RTime'], train_df['Glucose'], label='Glucose (Train)', color='red', alpha=1)   
        ax1.scatter(train_df['RTime'], train_df['Biomass'], label='Biomass (Train)', color='green', alpha=1)
        ax2.scatter(train_df['RTime'], train_df['Protein'], label='Protein (Train)', color='blue', alpha=1)
        ax3.scatter(train_df['RTime'], train_df['V'], label='Volume (Train)', color='purple', alpha=1)
    
    if net_df is not None:
        ax1.scatter(net_df['RTime'], net_df['Glucose'], label='Glucose (Predicted)', marker='x', color='red', alpha=0.5)
        ax1.scatter(net_df['RTime'], net_df['Biomass'], label='Biomass (Predicted)', marker='x', color='green', alpha=0.5)
        ax2.scatter(net_df['RTime'], net_df['Protein'], label='Protein (Predicted)', marker='x', color='blue', alpha=0.5)
        ax3.scatter(net_df['RTime'], net_df['V'], label='Volume (Predicted)', marker='x', color='purple', alpha=0.5)

    ax1.set_xlabel("Time (hours)")
    ax1.set_ylabel("Concentration")
    ax1.legend()
    
    ax2.set_xlabel("Time (hours)")
    ax2.set_ylabel("Concentration")
    ax2.legend()

    ax3.set_xlabel("Time (hours)")
    ax3.set_ylabel("Volume")
    ax3.legend()
        
    plt.show()
 
def get_feed(feeds: pd.DataFrame, time: float) -> float:
    for _, row in feeds.iterrows():
        start_time = row["Time"]
        end_time = row["Time"] + row["Duration"]
        if start_time <= time < end_time:
            return row["F"] / 1000
    return 0

def simulate(df: pd.DataFrame, feeds: pd.DataFrame, \
             mu_max, Ks, Yxs, c1, c2, plot: bool = True):
    
    mu_max = mu_max
    Ks = Ks
    Yxs = Yxs
    Sin = 1.43 * 200

    def alpha(t):
        return c1 * (1 - np.exp(-c2 * t**2))

    def system_ode(t, y):
        X, S, P, V = y
        mu = mu_max * S / (Ks + S)
        F = get_feed(feeds, t)
        dXdt = mu * X + F * X / V
        dSdt = - mu * X / Yxs + F * (Sin - S) / V
        dPdt = alpha(t) * X - P * F / V
        dVdt = F
        return [dXdt, dSdt, dPdt, dVdt]
    
    t_start, t_end = df['RTime'].min(), df['RTime'].max()
    t_span = (t_start, t_end)
    y0 = [df['Biomass'].iloc[0], df['Glucose'].iloc[0], \
          df['Protein'].iloc[0], df['V'].iloc[0]]

    t_eval = np.linspace(t_start, t_end, 10000)
    sol = solve_ivp(system_ode, t_span=t_span, \
        y0=y0, t_eval=t_eval)
    
    if plot:
        plot_simulation(sol.t, sol.y, full_df=df)

    for i in range(sol.y.shape[0]):
        sol.y[i][sol.y[i] < 0] = 0

    return sol

def get_predictions_df(net: nn.Module, df: pd.DataFrame):
    net_df = pd.DataFrame(columns=['RTime', 'Biomass', 'Glucose'])
    t_test = df['RTime'].values
    t_test = numpy_to_tensor(t_test)
    net_df["RTime"] = df["RTime"].values
    net_df["Biomass"] = net.forward(t_test).detach().cpu().numpy()[:, 0]
    net_df["Glucose"] = net.forward(t_test).detach().cpu().numpy()[:, 1]
    net_df["V"] = net.forward(t_test).detach().cpu().numpy()[:, 2]
    net_df["Protein"] = net.forward(t_test).detach().cpu().numpy()[:, 3]
    net_df.loc[net_df['Glucose'] < 0, 'Glucose'] = 0
    return net_df

In [159]:
FILENAME = '../data/data_processed.xlsx'
EXPERIMENT = 'BR04'

df, feeds = get_data_and_feed(FILENAME, EXPERIMENT)

# Only FED-BATCH-INDUCTION data
df = df[df['Process'] == 'FBI']
feeds = feeds[feeds['Induction']==1]

print(f'Dataset shape: {df.shape}')

Dataset shape: (6, 8)


In [160]:
def main(df: pd.DataFrame, num_epochs: int = 2500):
    t_start, t_end = df['RTime'].min(), df['RTime'].max()

    t_train = numpy_to_tensor(df['RTime'].values)
    Biomass_train = numpy_to_tensor(df['Biomass'].values)
    Glucose_train = numpy_to_tensor(df['Glucose'].values)
    V_train = numpy_to_tensor(df['V'].values)
    Protein_train = numpy_to_tensor(df['Protein'].values)
    u_train = torch.cat((Biomass_train, Glucose_train, V_train, Protein_train), 1)

    net = PINN(input_dim=1, output_dim=4, t_start=t_start, t_end=t_end)

    net = train(net, t_train, u_train, df, feeds, \
                num_epochs=num_epochs, verbose=True)

    net_df = get_predictions_df(net, df)     

    sol = simulate(df, feeds, net.mu_max.item(), net.K_s.item(), net.Y_xs.item(), \
               c1=net.c1.item(), c2=net.c2.item(), plot=False)

    title = f'c1 = {net.c1.item():.4f}, c2 = {net.c2.item():.4f}'
    plot_simulation(full_df=df, net_df=net_df, t=sol.t, y=sol.y, title=title)
    
    return net, net_df

In [161]:
def V(t):
    return 31/1000 * (t-9.933333) + 1.422819

df['V'] = V(df['RTime'])

In [166]:
i = 7
print(f'Running for {i} data points')
df_ = df.iloc[:i]
net, net_df = main(df_, num_epochs=2500)

Running for 7 data points


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 2/2500 [00:00<03:45, 11.09it/s]

X_data_loss: 204.44029235839844
S_data_loss: 1.0661698579788208
V_data_loss: 2.3699967861175537
P_data_loss: 0.008381408639252186
X_IC_loss: 181.38694763183594
S_IC_loss: 0.6497958898544312
V_IC_loss: 2.1081597805023193
P_IC_loss: 0.021655822172760963
tensor([-0.0291, -0.0324, -0.0351, -0.0380, -0.0412, -0.0438],
       grad_fn=<SelectBackward0>)


  1%|          | 22/2500 [00:03<07:16,  5.67it/s]


KeyboardInterrupt: 

In [ ]:
print(f'c1={net.c1.item()}, c2={net.c2.item()}')

c1=0.002346951048821211, c2=0.00561975734308362


In [ ]:
net_df

,RTime,Biomass,Glucose,V,Protein
0,9.933333,27.703373,0.024228,1.505409,0.110385
1,11.033333,30.653744,0.022353,1.664382,0.162012
2,11.966667,33.157093,0.020762,1.799267,0.205817
3,12.966667,35.839252,0.019058,1.943787,0.252751
4,14.050000,38.744926,0.017211,2.100350,0.303594
5,14.933333,41.114166,0.015705,2.228009,0.345051
